## Download code and data

In [ ]:
!git init
!git remote add origin https://github.com/dtungpka/AligningVTVSignlanguage.git
!git pull
!git checkout main -f
!git branch --set-upstream-to origin/main

If on colab, run the following cell to download the code and data. If running locally, you can download the data from [skeleton.zip](https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/EeqGCq4MsZtMm54TiKhx980BAPQSrwMjPX_hpMRaSMD-Uw?download=1) and [evaluation_pack.pkl](https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/ESIr97SZa6FNtG3P76ddu9EBk_eKCnVG-HHM71rjNVjKpQ?download=1)

In [ ]:
!wget -O skeletons.zip "https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/EeqGCq4MsZtMm54TiKhx980BAPQSrwMjPX_hpMRaSMD-Uw?download=1"
!unzip skeletons.zip
!wget -O evaluation_pack.pkl "https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/ESIr97SZa6FNtG3P76ddu9EBk_eKCnVG-HHM71rjNVjKpQ?download=1"

In [ ]:
import os
import train
import pandas as pd
import os
import glob
import pickle

## 1. Select signs

In [ ]:
from select_signs import main as select_signs_main
if not os.path.exists('pending_training.txt'):
    select_signs_main()
else:
    print('pending_training.txt exists. Skip select_signs')

You should download the `pending_training.txt` file to backup it

## 2. Training

In [ ]:
EPOCHS = 50
N_FRAMES = 5


signs_file = 'pending_training.txt'
if not os.path.exists(signs_file):
    print(f"File '{signs_file}' does not exist.")
    exit(1)
with open(signs_file, 'r') as f:
    signs_list = [line.strip() for line in f.readlines()]
train.process_signs(data_folder="skeletons", save_path="models", signs_list=signs_list, epochs=EPOCHS, n_frames=N_FRAMES)

In [ ]:
# Get list of all CSV files in the results folder
csv_files = glob.glob(os.path.join('results', 'training_results_*.csv'))

print(csv_files)
# Get the newest CSV file
newest_csv = max(csv_files, key=os.path.getctime)

# Read the newest CSV file into a pandas DataFrame
df_newest = pd.read_csv(newest_csv)

# Display the DataFrame interactively
df_newest

## 3. Evaluate on VTV

In [ ]:
if os.path.exists(os.path.join('results', 'results.csv')):
    os.remove(os.path.join('results', 'results.csv'))
from evaluate import process_evaluation_pack

evaluation_pack_path = 'evaluation_pack.pkl'
checkpoint_path = 'models'
save_path = 'results'
frame_count = 5 #equal to N_FRAMES
delta = 120 #window size to find the sign

# Load evaluation pack and process
with open(evaluation_pack_path, 'rb') as f:
    evaluation_pack = pickle.load(f)

process_evaluation_pack(evaluation_pack, checkpoint_path, save_path, frame_count, delta)

In [ ]:
# Get list of all CSV files in the results folder
csv_files = glob.glob(os.path.join('results', 'results.csv'))

print(csv_files)
# Get the newest CSV file
newest_csv = max(csv_files, key=os.path.getctime)

# Read the newest CSV file into a pandas DataFrame
df_newest = pd.read_csv(newest_csv)

# Display the DataFrame interactively
df_newest